In [1]:
! git clone -b dev https://github.com/motazsaad/infant-cry-care.git

Cloning into 'infant-cry-care'...
remote: Enumerating objects: 7889, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7889 (delta 1), reused 6 (delta 1), pack-reused 7882
Receiving objects: 100% (7889/7889), 474.34 MiB | 30.36 MiB/s, done.
Resolving deltas: 100% (512/512), done.
Checking out files: 100% (14949/14949), done.


In [4]:
import pandas as pd 
import glob

classes=['Hungry','Sleepy','Uncomfortable','asphyxia','burp up','lower gas','normal','pain']
test_files = glob.glob('/content/infant-cry-care/infant-cry-dataset/partitioned-data/val/*/*.wav', recursive=True)
test_labels = [fname.split('/')[-2] for fname in test_files]
test_df = pd.DataFrame({'wav_file': test_files, 'label': test_labels})
test_df.head()

,wav_file,label
0,/content/infant-cry-care/infant-cry-dataset/pa...,normal
1,/content/infant-cry-care/infant-cry-dataset/pa...,normal
2,/content/infant-cry-care/infant-cry-dataset/pa...,normal
3,/content/infant-cry-care/infant-cry-dataset/pa...,normal
4,/content/infant-cry-care/infant-cry-dataset/pa...,normal


In [13]:
! pip install audio-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 700 kB/s 
     |████████████████████████████████| 155 kB 13.7 MB/s 
     |████████████████████████████████| 52 kB 972 kB/s 
     |████████████████████████████████| 489 kB 57.6 MB/s 
  Created wheel for bitstruct: filename=bitstruct-8.15.1-cp38-cp38-linux_x86_64.whl size=61651 sha256=39689ac1245f66efa7a1e23b5d241a08f3e3d7c8bbc4caa4c071eeabca1869b9
  Stored in directory: /root/.cache/pip/wheels/aa/78/79/94ce5423af175c01170f7dac3453eb2a98a1d72d55ca9ec401
Successfully built bitstruct
  Attempting uninstall: attrs
    Found existing installation: attrs 22.1.0
    Uninstalling attrs-22.1.0:
      Successfully uninstalled attrs-22.1.0
  Attempting uninstall: more-itertools
    Found existing installation: more-itertools 9.0.0
    Uninstalling more-itertools-9.0.0:
      Successfully uninstalled more-itertools-9.0.0


In [10]:
# path = '../tests/test_files/A101_1.WAV'
wav_file = test_df.iloc[0].wav_file
wav_file

'/content/infant-cry-care/infant-cry-dataset/partitioned-data/val/normal/0052318000.wav'

In [16]:
import audio_metadata
metadata = audio_metadata.load(wav_file)
metadata

<WAVE({
    'filepath': '/content/infant-cry-care/infant-cry-dataset/partitioned-data/val/normal/0052318000.wav',
    'filesize': '15.70 KiB',
    'pictures': [],
    'streaminfo': <WAVEStreamInfo({
        'audio_format': <WAVEAudioFormat.PCM>,
        'bit_depth': 16,
        'bitrate': '128 Kbps',
        'channels': 1,
        'duration': '00:01',
        'sample_rate': '8.0 KHz',
    })>,
    'tags': <RIFFTags({'ISFT': ['Lavf57.83.100']})>,
})>

In [17]:
type(metadata)

audio_metadata.formats.wave.WAVE

In [24]:
def get_wav_info(df):
  for i, row in df.iterrows():
    metadata = audio_metadata.load(row['wav_file'])
    df.loc[i, 'filesize'] = metadata.filesize
    df.loc[i, 'bit_depth'] = metadata.streaminfo.bit_depth
    df.loc[i, 'bitrate'] = metadata.streaminfo.bitrate
    df.loc[i, 'channels'] = metadata.streaminfo.channels
    df.loc[i, 'duration'] = metadata.streaminfo.duration
    df.loc[i, 'sample_rate'] = metadata.streaminfo.sample_rate


In [25]:
get_wav_info(test_df)

In [26]:
test_df

,wav_file,label,filesize,bit_depth,bitrate,channels,duration,sample_rate
0,/content/infant-cry-care/infant-cry-dataset/pa...,normal,16078.0,16.0,128000.0,1.0,1.000000,8000.0
1,/content/infant-cry-care/infant-cry-dataset/pa...,normal,22138.0,16.0,176400.0,1.0,1.001995,11025.0
2,/content/infant-cry-care/infant-cry-dataset/pa...,normal,22138.0,16.0,176400.0,1.0,1.001995,11025.0
3,/content/infant-cry-care/infant-cry-dataset/pa...,normal,16078.0,16.0,128000.0,1.0,1.000000,8000.0
4,/content/infant-cry-care/infant-cry-dataset/pa...,normal,22138.0,16.0,176400.0,1.0,1.001995,11025.0
...,...,...,...,...,...,...,...,...
1772,/content/infant-cry-care/infant-cry-dataset/pa...,lower gas,228166.0,16.0,176400.0,1.0,10.345669,11025.0
1773,/content/infant-cry-care/infant-cry-dataset/pa...,lower gas,33888.0,16.0,256000.0,1.0,1.056563,16000.0
1774,/content/infant-cry-care/infant-cry-dataset/pa...,lower gas,22872.0,16.0,176400.0,1.0,1.035283,11025.0
1775,/content/infant-cry-care/infant-cry-dataset/pa...,lower gas,20570.0,16.0,176400.0,1.0,0.930884,11025.0


In [28]:
test_df.describe()

,filesize,bit_depth,bitrate,channels,duration,sample_rate
count,1.777000e+03,1777.0,1.777000e+03,1777.000000,1777.000000,1777.000000
mean,4.625992e+04,16.0,1.933819e+05,1.016320,1.827236,11465.095667
std,1.056018e+05,0.0,1.493511e+05,0.126737,2.159836,4476.859023
min,4.512000e+03,16.0,1.280000e+05,1.000000,0.202630,8000.000000
25%,1.961000e+04,16.0,1.764000e+05,1.000000,0.911111,11025.000000
50%,2.213800e+04,16.0,1.764000e+05,1.000000,1.001995,11025.000000
75%,2.213800e+04,16.0,1.764000e+05,1.000000,1.001995,11025.000000
max,2.095036e+06,16.0,1.536000e+06,2.000000,10.912925,48000.000000


In [30]:
test_df.describe().bitrate['max']

1536000.0

In [31]:
test_df.describe().bitrate['min']

128000.0